In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pendulum
df = pd.read_sql_table('tbl_envolvidos', 'sqlite:///gdo.db',
                       columns =['NREDS', 'COD_DIAO', 'T_C', 'COD_DELITO', 'T_C_DELITO','GRAU_LESAO','GRUPO_TIPO_ENVOL',
                                 'DATA_FATO', 'MUNICIPIO','CIA', 'UEOP', 'QTDE_ENVOL'])

In [18]:
### FILTRO PARA RETORNAR  APENAS  AS VITIMAS DE  HOMICIDIO CONSUMADO CONFORME CRTITERIOS GDO ###
date_init_before = pendulum.now().subtract(years=1, months=(datetime.today() - timedelta(days=1)).month-1, days=(datetime.today() - timedelta(days=1)).day).strftime('%Y-%m-%d')
date_ende_before = pendulum.now().subtract(years=1, days=1).strftime('%Y-%m-%d')

date_init_actual = pendulum.now().subtract(months=(datetime.today() - timedelta(days=1)).month-1, days=(datetime.today() - timedelta(days=1)).day).strftime('%Y-%m-%d')
date_ende_actual = pendulum.now().subtract(days=1).strftime('%Y-%m-%d')

#date_ende_actual = (datetime.today()- timedelta(days=1)).strftime('%Y-%m-%d') # Infome da  data final da pesquisa

#datetime.strptime(date_init, '%Y-%m-%d')

#datetime.strptime(date_end, '%Y-%m-%d')

df.loc[:,'ANO'] = df.loc[:,'DATA_FATO'].dt.year

df.loc[:,'MES'] = df.loc[:,'DATA_FATO'].dt.month



df_hc = df.loc[
    (
        (df['COD_DIAO'] == 'B01121')    & (df['T_C'] == 'CONSUMADO')
                  
                                        |
    
        (df['COD_DELITO'] == 'B01121')  & (df['T_C_DELITO'] == 'CONSUMADO')
        
                                        |
        (df['NREDS'] == '2020-017068849-001')# NESTA LINHA  EU INCLUO OS REDS QUE NÃO SATISFAZEM OS CRITERIOS DA GDO 
        #MAS  QUE  FORAM INCLUIDOS POR AUDITORIA
        
    )
    
                                        &
    
    (df['GRAU_LESAO'] == 'FATAL')       & (df['GRUPO_TIPO_ENVOL'] == 'VITIMA')
    
                                        & 
    (
    (df['DATA_FATO']>=date_init_before) & (df['DATA_FATO']<=date_ende_before)
    
                                        |
        
    (df['DATA_FATO']>=date_init_actual) & (df['DATA_FATO']<=date_ende_actual)
        
    )
]




#df_hc

In [15]:
df_hcc = pd.pivot_table(df_hc,
               index=['UEOP','MUNICIPIO'], 
               values=['QTDE_ENVOL'], 
               #columns=['ANO', 'MES'],
               #columns=['ANO'],
               columns=['MES'],
               fill_value=0,
               aggfunc=np.sum, 
               #margins=True, 
               #margins_name='Total Geral'
               #observed =       

              )
try:
    
    df_hcc['VAR']  = (df_hcc[('QTDE_ENVOL',    2020)] *100 / df_hcc[('QTDE_ENVOL',    2019)] - 100).round(1)
        
except ZeroDivisionError:
    
    df_hcc['VAR'] = 0

df_hcc.replace([np.inf, -np.inf], 100, inplace=True)

#df_hcc['VAR'].astype(float)

df_hcc



KeyError: ('QTDE_ENVOL', 2020)

In [11]:
def farol_colors(val):
    feliz = '&#128578'
    normal = '&#x1f610'
    triste = '&#128577'
    if val < 0 :
        color = 'green'
    elif val == 0:
        color = 'yellow'
    elif val > 0:
        color = 'red'
    else:
        color = 'orange'
    return (
        '''                
        background-color: {};
        font-size: 14px;
        text-align: center;        
        '''.format(color)
    )

df_hcc_color = df_hcc.style.applymap(farol_colors, subset=['VAR'])



from IPython.display import display, HTML
title = display(HTML(' <span style="color:orange"><font size=0pt><h1>VITIMAS DE HOMICIDIOS CONSUMADOS</h1> </span>  ')
       
       )

html = df_hcc_color.render()

import imgkit
path_wkthmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)


imgkit.from_string(html, 'Vitímas de Homicídio Consumado.jpg',config=config)





Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

In [11]:
df_hcc.style.set_table_styles( 
[ 
 {'selector': 'th', 
  'props': [('background', '# 606060'),  
            ('color', 'green'), ]}, 
 {'selector': 'td', 
  'props': [('color', 'blue')]}, 
]) 


In [20]:
pd.pivot_table(df_hc,
               index=['UEOP','MUNICIPIO'], 
               values=['QTDE_ENVOL'], 
               columns=['ANO', 'MES'],
               #columns=['ANO'],
               #columns=['MES'],
               fill_value=0,
               aggfunc=np.sum, 
               #margins=True, 
               #margins_name='Total Geral'
               #observed =       

              )

QTDE_ENVOL                               \
ANO                                         2019                   2020         
MES                                            1  2  3  4  5  6  7    1  2  3   
UEOP           MUNICIPIO                                                        
07º BPM        BOM DESPACHO                    0  2  2  1  0  0  0    2  1  0   
               DORES DO INDAIA                 0  0  0  0  0  0  1    0  0  0   
               LAGOA DA PRATA                  1  0  0  2  0  0  0    0  0  0   
               LUZ                             1  0  0  0  0  0  0    0  0  0   
               MARTINHO CAMPOS                 0  0  0  0  0  0  0    0  1  0   
               PEDRA DO INDAIA                 0  0  0  0  0  1  0    0  0  0   
               POMPEU                          4  2  1  1  3  0  2    0  2  3   
               SANTO ANTONIO DO MONTE          0  0  0  1  0  0  0    0  0  0   
19ª CIA PM IND IGARATINGA                      0  0  0  0  0  0  0    0  0  0   
               PAPAGAIOS                       1  0  0  1  0  0  0    0  0  0   
               PARA DE MINAS                   0  1  3  1  0  0  1    1  1  4   
               PEQUI                           0  1  0  0  0  0  0    0  0  0   
23º BPM        CARMO DO CAJURU                 0  1  0  0  0  0  0    0  0  0   
               CLAUDIO                         0  0  0  0  0  0  0    0  0  0   
               DIVINOPOLIS                     3  4  6  5  3  2  2    3  3  4   
               ITATIAIUCU                      1  0  0  1  0  0  0    0  1  0   
               ITAUNA                          2  0  0  0  1  1  2    2  4  1   
               SAO GONCALO DO PARA             0  1  0  0  0  1  0    0  0  0   
60º BPM        ARAUJOS                         0  1  0  0  2  0  0    0  0  0   
               LEANDRO FERREIRA                0  0  1  0  0  0  0    0  0  0   
               NOVA SERRANA                    3  1  0  2  1  0  0    1  1  0   
               PITANGUI                        1  0  1  0  0  1  0    0  2  0   
63º BPM        BAMBUI                          0  0  0  0  0  0  0    0  1  0   
               CORREGO FUNDO                   0  0  0  0  0  0  0    0  0  1   
               FORMIGA                         2  0  0  1  1  1  1    2  0  0   
               IGUATAMA                        0  0  0  0  0  0  1    1  0  0   
               ITAPECERICA                     1  0  0  0  0  0  0    0  0  0   
               PIMENTA                         0  0  0  0  0  0  0    1  0  0   
               SAO SEBASTIAO DO OESTE          0  0  0  0  0  2  0    1  0  0   
               TAPIRAI                         0  0  0  0  0  0  0    0  0  0   

                                                   
ANO                                                
MES                                    4  5  6  7  
UEOP           MUNICIPIO                           
07º BPM        BOM DESPACHO            1  0  1  0  
               DORES DO INDAIA         0  0  0  0  
               LAGOA DA PRATA          0  1  0  0  
               LUZ                     0  0  0  0  
               MARTINHO CAMPOS         0  1  0  0  
               PEDRA DO INDAIA         0  0  0  0  
               POMPEU                  2  1  2  1  
               SANTO ANTONIO DO MONTE  0  0  0  0  
19ª CIA PM IND IGARATINGA              0  0  1  0  
               PAPAGAIOS               0  0  2  0  
               PARA DE MINAS           0  0  1  0  
               PEQUI                   0  0  1  0  
23º BPM        CARMO DO CAJURU         0  0  1  0  
               CLAUDIO                 2  0  0  0  
               DIVINOPOLIS             1  1  2  1  
               ITATIAIUCU              0  0  0  0  
               ITAUNA                  1  0  1  0  
               SAO GONCALO DO PARA     0  0  0  0  
60º BPM        ARAUJOS                 0  0  0  0  
               LEANDRO FERREIRA        0  0  0  0  
               NOVA SERRANA        